# The Eye Of The Typer Dataset

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from pprint import pprint

import polars as pl

from the_eye_of_the_typer import *

In [ ]:
df = read_participant_characteristics()
df

In [ ]:
participants = [Participant.create(**row) for row in df.iter_rows(named=True)]

print("Participants:", len(participants), end="\n\n")
pprint(participants[0])

In [ ]:
p = participants[0]

In [ ]:
p.tobii_gaze_predictions

In [ ]:
rerun(p)